In [1]:
!pip install vectorbt
!pip install condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ta-lib
!pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 12.6 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for vectorbt: filename=vectorbt-0.26.2-py3-none-any.whl size=527327 sha256=8c2e2613701ef09146fe70358cd45656a95089f85b7c00394c11d652f9b95f0f
  Stored in directory: /root/.cache/pip/wheels/35/a3/d2/ae1a8cbf659fffc7d1890123df1a7fa2e7f6df587c1dca71cd
Successfully built vectorbt
  Attempting uninstall: numpy
    Found existing installation: numpy 

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: | / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ta-lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    libblas-3.9.0              |24_linux64_openblas          15 KB  conda-forge
    libcbla

In [ ]:
!pip uninstall -y cffi
!pip uninstall -y _cffi_backend
!pip install cffi==1.17.0
import cffi
import _cffi_backend
print(cffi.__version__)
print(_cffi_backend.__version__)

Found existing installation: cffi 1.17.0
Uninstalling cffi-1.17.0:
  Successfully uninstalled cffi-1.17.0
ERROR: Invalid requirement: '_cffi_backend'
  Using cached cffi-1.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached cffi-1.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (445 kB)
1.17.1
1.17.0


In [ ]:
import vectorbt as vbt
import talib as ta
import requests
import json
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import time
import datetime




In [ ]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            # print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }
        # print(params)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:

            print(f"Failed to fetch data. Status code: {response.json()}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "timestamp", "open", "high", "low", "close", "volume",
          "Close Time", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]
      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # # Convert timestamp columns to datetime
      # df["timestamp"] = pd.to_datetime(df['timestamp'])
      # df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # # Convert numeric columns to float
      # numeric_columns = ["open", "high", "low", "close", "volume", "Quote Asset Volume",
      #                   "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      # df[numeric_columns] = df[numeric_columns].astype(float)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [ ]:
startTime = datetime.datetime(2023, 1, 1)
endTime = datetime.datetime.now()

In [ ]:
timeperiod = '1h'
symbol = "btcusdt"
downloadedData = GetBinanceData.get_data(symbol, timeperiod, startDate=startTime, endDate=endTime)


60000


In [ ]:
df = GetBinanceData.data_to_dataframe(downloadedData)  # 1-minute data
df = df.apply(pd.to_numeric, errors='coerce')

In [ ]:
timeperiodLower = '15m'
symbolLower = "btcusdt"
downloadedDataLowerTF = GetBinanceData.get_data(symbolLower, timeperiodLower, startDate=startTime, endDate=endTime)

15000


In [ ]:
# lowerTf_df
lowerTf_df = GetBinanceData.data_to_dataframe(downloadedData)  # 1-minute data
lowerTf_df = lowerTf_df.apply(pd.to_numeric, errors='coerce')

In [ ]:
def calculate_fvg(candles_df):
    # Initialize columns for FVG results
    candles_df['FVG_Type'] = ''

    # Find bullish FVGs
    bullish_conditions = (candles_df['low'].shift(-2) > candles_df['high'].shift(2)) & \
                         (candles_df['close'].shift(-1) > candles_df['high'].shift(2))

    candles_df.loc[bullish_conditions, 'FVG_Type'] = 'Bullish'

    # Find bearish FVGs
    bearish_conditions = (candles_df['high'].shift(-2) < candles_df['low'].shift(2)) & \
                         (candles_df['close'].shift(-1) < candles_df['low'].shift(2))

    candles_df.loc[bearish_conditions, 'FVG_Type'] = 'Bearish'

    return candles_df



# Calculate FVGs
result_df = calculate_fvg(df)

# Count of Bullish and Bearish FVGs
bullish_count = (result_df['FVG_Type'] == 'Bullish').sum()
bearish_count = (result_df['FVG_Type'] == 'Bearish').sum()

# Latest values of FVGs
latest_fvg = result_df[result_df['FVG_Type'].ne('')]


print("Last 10 rows of the Resulting DataFrame:")
i = df.tail(10)
i.head()

Last 10 rows of the Resulting DataFrame:


,timestamp,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore,FVG_Type,FVG_Value,prev_close,prev_high,prev_low,prev_close_2,prev_high_2,prev_low_2
15246,1727438400000,65403.82,65858.34,65353.70,65649.72,0.27134,1727441999999,17815.878169,132,0.08902,5839.518072,0,,None,65403.82,65598.76,65358.90,65544.74,65784.69,65474.21
15247,1727442000000,65720.41,65799.97,65389.91,65610.31,0.40797,1727445599999,26767.924395,146,0.32985,21644.932827,0,Bullish,None,65649.72,65858.34,65353.70,65403.82,65598.76,65358.90
15248,1727445600000,65692.37,66398.77,65692.37,66281.21,4.53759,1727449199999,299947.648711,501,2.89024,190914.608041,0,Bullish,None,65610.31,65799.97,65389.91,65649.72,65858.34,65353.70
15249,1727449200000,66281.60,66370.00,65968.70,66043.69,1.26948,1727452799999,84029.074409,252,0.37539,24850.832474,0,,None,66281.21,66398.77,65692.37,65610.31,65799.97,65389.91
15250,1727452800000,65973.17,66324.51,65898.31,66216.48,0.39535,1727456399999,26103.895103,274,0.19712,13026.621512,0,,None,66043.69,66370.00,65968.70,66281.21,66398.77,65692.37


In [ ]:

lowerTf_df['prev_high'] = lowerTf_df['high'].shift(1)
lowerTf_df['prev_low'] = lowerTf_df['low'].shift(1)
lowerTf_df['prev_close'] = lowerTf_df['close'].shift(1)

# Helper columns to define patterns
lowerTf_df['first_2_up'] = lowerTf_df['high'] > lowerTf_df['prev_high']  # 2 Up (Bullish)
lowerTf_df['first_2_down'] = lowerTf_df['low'] < lowerTf_df['prev_low']  # 2 Down (Bearish)
lowerTf_df['inside_1'] = (lowerTf_df['high'] < lowerTf_df['prev_high']) & (lowerTf_df['low'] > lowerTf_df['prev_low'])  # Inside bar (1)
lowerTf_df['outside_3'] = (lowerTf_df['high'] > lowerTf_df['prev_high']) & (lowerTf_df['low'] < lowerTf_df['prev_low'])  # Outside bar (3)

# Pattern 1: 2-1-2U Bullish Continuation
lowerTf_df['second_2_up'] = lowerTf_df['high'] > lowerTf_df['high'].shift(1)
lowerTf_df['2-1-2U_bullish_continuation'] = lowerTf_df['first_2_up'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_up']

# Pattern 2: 2-1-2D Bearish Continuation
lowerTf_df['second_2_down'] = lowerTf_df['low'] < lowerTf_df['low'].shift(1)
lowerTf_df['2-1-2D_bearish_continuation'] = lowerTf_df['first_2_down'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_down']

# Pattern 3: 2-1-2U Bullish Reversal
lowerTf_df['2-1-2U_bullish_reversal'] = lowerTf_df['first_2_down'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_up']

# Pattern 4: 2-1-2D Bearish Reversal
lowerTf_df['2-1-2D_bearish_reversal'] = lowerTf_df['first_2_up'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_down']

# Pattern 5: 3-1-2U Bullish Reversal
lowerTf_df['3-1-2U_bullish_reversal'] = lowerTf_df['outside_3'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_up']

# Pattern 6: 3-1-2D Bearish Reversal
lowerTf_df['3-1-2D_bearish_reversal'] = lowerTf_df['outside_3'].shift(2) & lowerTf_df['inside_1'].shift(1) & lowerTf_df['second_2_down']

# Pattern 7: 2D-2U Reversal Bullish
lowerTf_df['2D-2U_bullish_reversal'] = lowerTf_df['first_2_down'].shift(1) & lowerTf_df['first_2_up']

# Pattern 8: 2U-2D Reversal Bearish
lowerTf_df['2U-2D_bearish_reversal'] = lowerTf_df['first_2_up'].shift(1) & lowerTf_df['first_2_down']

# Pattern 9: 3-2D-2U Bullish Reversal
lowerTf_df['3-2D-2U_bullish_reversal'] = lowerTf_df['outside_3'].shift(2) & lowerTf_df['first_2_down'].shift(1) & lowerTf_df['first_2_up']

# Pattern 10: 3-2U-2D Bearish Reversal
lowerTf_df['3-2U-2D_bearish_reversal'] = lowerTf_df['outside_3'].shift(2) & lowerTf_df['first_2_up'].shift(1) & lowerTf_df['first_2_down']

# Pattern 11: PMG Bearish (multiple 2D)
lowerTf_df['PMG_bearish'] = lowerTf_df['first_2_down'].rolling(3).sum() == 3  # 3 consecutive 2 downs

# Pattern 12: PMG Bullish (multiple 2U)
lowerTf_df['PMG_bullish'] = lowerTf_df['first_2_up'].rolling(3).sum() == 3  # 3 consecutive 2 ups

# Pattern 13: Rev Strat 1-2-2 Bullish
lowerTf_df['RevStrat_1-2-2_bullish'] = lowerTf_df['inside_1'].shift(2) & lowerTf_df['first_2_down'].shift(1) & lowerTf_df['first_2_up']

# Pattern 14: Rev Strat 1-2U-2D Bearish
lowerTf_df['RevStrat_1-2U-2D_bearish'] = lowerTf_df['inside_1'].shift(2) & lowerTf_df['first_2_up'].shift(1) & lowerTf_df['first_2_down']

# Select the columns with pattern detection
patterns_lowerTf_df = lowerTf_df[['open', 'high', 'low', 'close', '2-1-2U_bullish_continuation', '2-1-2D_bearish_continuation',
                                  '2-1-2U_bullish_reversal', '2-1-2D_bearish_reversal', '3-1-2U_bullish_reversal',
                                  '3-1-2D_bearish_reversal', '2D-2U_bullish_reversal', '2U-2D_bearish_reversal',
                                  '3-2D-2U_bullish_reversal', '3-2U-2D_bearish_reversal', 'PMG_bearish', 'PMG_bullish',
                                  'RevStrat_1-2-2_bullish', 'RevStrat_1-2U-2D_bearish']]

def determine_type(row):
    # Check for Bullish Patterns
    if row['2-1-2U_bullish_continuation']:
        return 'Bullish'
    elif row['2-1-2U_bullish_reversal']:
        return 'Bullish'
    elif row['3-1-2U_bullish_reversal']:
        return 'Bullish'
    elif row['2D-2U_bullish_reversal']:
        return 'Bullish'
    elif row['2U-2D_bearish_reversal']:
        return 'Bullish'
    elif row['3-2D-2U_bullish_reversal']:
        return 'Bullish'
    elif row['RevStrat_1-2-2_bullish']:
        return 'Bullish'
    elif row['PMG_bullish']:
        return 'Bullish'

    # Check for Bearish Patterns
    if row['2-1-2D_bearish_continuation']:
        return 'Bearish'
    elif row['2-1-2D_bearish_reversal']:
        return 'Bearish'
    elif row['3-1-2D_bearish_reversal']:
        return 'Bearish'
    elif row['3-2U-2D_bearish_reversal']:
        return 'Bearish'
    elif row['PMG_bearish']:
        return 'Bearish'

    # If no patterns matched, return empty string
    return ''

lowerTf_df['Type'] = lowerTf_df.apply(determine_type, axis=1)
# Count occurrences of each type
type_counts = lowerTf_df['Type'].value_counts()

# Display the counts
print(type_counts)
len(lowerTf_df)

Type
Bullish    7888
           5453
Bearish    1915
Name: count, dtype: int64


15256